In [1196]:
# -*- coding: utf8 -*-
from os.path import expanduser
home = expanduser("~")
repertoire=home+"/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/Kalaba-Project/16-K1"
serie=repertoire+"/"
variante="-Corr"
variante=""
complementPhrases=""
#########################IMPORTS############################################
import codecs, optparse
import re, random
import sys,os,time
import string
import yaml
import ParFuMor as PFM
from ParFuMor import *
import pickle

### Javascript from :
https://stackoverflow.com/questions/12544056/how-to-i-get-the-current-ipython-notebook-name

In [1197]:
%%javascript
var kernel = IPython.notebook.kernel;
var thename = window.document.getElementById("notebook_name").innerHTML;
var command = "theNotebook = " + "'"+thename+"'";
kernel.execute(command);

<IPython.core.display.Javascript object>

In [1198]:
#########################VARIABLES##########################################
version=os.path.basename("__file__")
time_stamp='%s' % time.strftime("%y%m%d-%H%M")
debug=0
debug_now=1
#casNombreDet=True
separateurPhonoCloze=" "
separateurMots=separateurPhonoCloze
#separateurMots=" "
marqueurCommentaire="%"
print_no=True
print_taches=False
print_coffee=False
print_commands=True
print_ortho=True
print_phono=True
print_glose=False
print_radicaux=False
#if separateurMots=="":
#    print_glose=False
if variante=="-Corr":
    print_taches=False
    print_coffee=False
    print_commands=True
    print_ortho=True
    print_phono=True
    print_glose=True    
if print_glose:
    separateurMots=" "
if print_glose+print_ortho+print_phono>1:
    print_phrases=True
else:
    print_phrases=False
print_lexique=True
print_cloze=True
print_racines=True
no_form="***"
# no_grapho=['dormir', 'lit']
# no_phono=['gros', 'coussin']
no_grapho=['petit','Nabil',"sur"]
no_phono=["grand","autruche",'dans']
phono_no=u"XXXXX"
grapho_no=u"XXXXX"

In [1199]:
print_glose+print_ortho+print_phono

2

In [1200]:
prononciationBegin=[
    "\\begin{center}",
        "\\begin{tabular}{lcc}",
        "\\toprule",
        u"Graphie & Prononciation & Mot français \\\\",
        "\\midrule"
        ]
prononciationEnd=[
        "\\bottomrule",
        "\\end{tabular}",
    "\\end{center}"
]

In [1201]:
with open(serie+"Gloses.yaml", 'r') as stream:
    gloses=yaml.load(stream)
with open(serie+"Stems.yaml", 'r') as stream:
    stems=yaml.load(stream)
with open(serie+"Phonology.yaml", 'r') as stream:
    phonology=yaml.load(stream)
with open(serie+"MorphoSyntax.yaml", 'r') as stream:
    morphosyntax=yaml.load(stream)
with open(serie+"Clozes.txt", 'r') as stream:
    clozeLines=stream.readlines()

discrimineur=[]
discriminant={}
for line in clozeLines:
    line=line.strip()
    if not line.startswith("#"):
        elementsCloze=line.split(";")
        discriminant[elementsCloze[0]]=";".join([element for element in elementsCloze[1:] if element!=elementsCloze[5]])

In [1202]:
#discriminant

In [1203]:
defaultCols=2
defaultLong=0
maxChunk=48
dimensionsTableaux={i:{} for i in gloses}
for categorie in gloses:
    if "Dimensions" in morphosyntax:
        if "maxChunk" in morphosyntax["Dimensions"]:
            maxChunk=morphosyntax["Dimensions"]["maxChunk"]
        print maxChunk
        if categorie in morphosyntax["Dimensions"] and morphosyntax["Dimensions"][categorie]:
#            print categorie
            if "cols" in morphosyntax["Dimensions"][categorie]:
                dimensionsTableaux[categorie]["cols"]=morphosyntax["Dimensions"][categorie]["cols"]
            else:
                dimensionsTableaux[categorie]["cols"]=defaultCols
            if "long" in morphosyntax["Dimensions"][categorie]:
                dimensionsTableaux[categorie]["long"]=morphosyntax["Dimensions"][categorie]["long"]
            else:
                dimensionsTableaux[categorie]["long"]=defaultLong
        else:
            dimensionsTableaux[categorie]["cols"]=defaultCols
            dimensionsTableaux[categorie]["long"]=defaultLong
    else:
        dimensionsTableaux[categorie]["cols"]=defaultCols
        dimensionsTableaux[categorie]["long"]=defaultLong
if print_radicaux:
    nomTableaux="Tableaux-Gloses.yaml"
else:
    nomTableaux="Tableaux.yaml"    
with open(serie+nomTableaux, 'r') as stream:
    tableaux=yaml.load(stream)
with open(serie+"Hierarchie-S2.pkl", 'rb') as input:
   PFM.hierarchieCF = pickle.load(input)
with open(serie+"Lexique-S2.pkl", 'rb') as input:
   PFM.lexique = pickle.load(input)
with open(serie+"Regles-S2.pkl", 'rb') as input:
   PFM.regles = pickle.load(input)

29
29
29
29
29


In [1204]:
[morphosyntax["Contractions"][x] for x in morphosyntax["Contractions"]]

[[u'\xe0', 'les'], ['de', 'le'], ['de', 'les'], [u'\xe0', 'le'], ['de']]

#### Définition des entêtes

In [1205]:
#########################CONSTANTS##########################################
head = [
"\\begin{tabular}[t]{|l|l|l|}",
"\\addlinespace[-1.0em]\\hline",
"Mot & Roman & Glose  \\\\",
"\\hline\\strutgh{14pt}%"
]
head_n = [
"\\begin{tabular}[t]{|l|c|c|c|}",
"\\addlinespace[-1.0em]\\hline",
"Nom & Genre & C\\indice{1}C\\indice{2}C\\indice{3} & V\\indice{L}  \\\\",
"\\hline\\strutgh{14pt}%"
]
head_v = [
"\\begin{tabular}[t]{|l|c|c|}",
"\\addlinespace[-1.0em]\\hline",
"Verbe & Type & C\\indice{1}C\\indice{2}C\\indice{3} \\\\",
"\\hline\\strutgh{14pt}%"
]

In [1206]:
tail = [
"\\hline"
"\\end{tabular}\\\\"
]

#### Définition des structures pour impression

In [1207]:
def accumulerMots(mot):
    accumulateur.append(mot)
    return
def ajouterExemple(exemple,printBool=False):
    if printBool:
        print exemple
    exemples.append(exemple.strip())
    del accumulateur[:]
    return
def ajouterVocabulaire(terme,printBool=False):
    if printBool:
        print terme
    vocabulaire.append(terme.strip())
    return

#### Définition des segments

In [1208]:
consonnes=phonology["consonnes"]
voyelles=phonology["voyelles"]
gabarits=phonology["gabarits"]
derives=phonology["derives"]
nom_classe=phonology["nom_classe"]
nom_apo=phonology["apophonies"]

nom_mut=phonology["mutations"]

#### Définition des catégories

In [1209]:
sujetVerbe=[]
if "Cas" in gloses["NOM"]:
    if "Acc" in gloses["NOM"]["Cas"]:
        sujetVerbe.append("Nom")
    if "Erg" in gloses["NOM"]["Cas"]:
        sujetVerbe.append("Abs")

In [1210]:
#sujetVerbe

In [1211]:
attributsMots=set()
valAttributs={}
for categorie in gloses:
    if gloses[categorie]:
        for element in gloses[categorie]:
            attributsMots.add(element)
            if gloses[categorie][element]:
                if not element in valAttributs:
                    valAttributs[element]=[]
                for attribut in gloses[categorie][element]:
                    if not attribut in valAttributs[element]:
                        valAttributs[element].append(attribut)

In [1212]:
attributsMots

{'Genre', 'Nombre', 'Pers', 'Temps'}

In [1213]:
boolAttribut={}
flexCategories=[element for element in gloses if gloses[element]]
for element in flexCategories:
    boolAttribut[element]={key:key in gloses[element] for key in attributsMots}
flexCategories

['VER', 'NOM', 'DET', 'ADJ']

In [1214]:
boolAttribut

{'ADJ': {'Genre': True, 'Nombre': True, 'Pers': False, 'Temps': False},
 'DET': {'Genre': True, 'Nombre': True, 'Pers': False, 'Temps': False},
 'NOM': {'Genre': True, 'Nombre': True, 'Pers': False, 'Temps': False},
 'VER': {'Genre': False, 'Nombre': False, 'Pers': True, 'Temps': True}}

attributsDetAdjNom=["Genre","Nombre","Cas"]
valAttribut={}
for attribut in attributsDetAdjNom:
    if attribut in gloses["N"]:
        valAttribut[attribut]=gloses["N"][attribut]
    elif attribut in gloses["ADJ"]:
        valAttribut[attribut]=gloses["ADJ"][attribut]
    elif attribut in gloses["DET"]:
        valAttribut[attribut]=gloses["DET"][attribut]
    else:
        valAttribut[attribut]=[]
boolAttribut={}
for element in ["DET","ADJ","N"]:
    boolAttribut[element]={key:key in gloses[element] for key in attributsDetAdjNom}

In [1215]:
#types=gloses["V"]["CF"]
if "Cas" in morphosyntax:
    casSyntagmes=morphosyntax["Cas"]
else:
    casSyntagmes=""
lexiquePrepositions=[stems["PREP"][x][0] for x in stems["PREP"]]
casPreposition={}
for preposition in lexiquePrepositions:
    prep=preposition.upper()
    if casSyntagmes and prep in casSyntagmes:
        casPreposition[preposition]=casSyntagmes[prep].capitalize()
    elif casSyntagmes and "PREP" in casSyntagmes:
        casPreposition[preposition]=casSyntagmes["PREP"].capitalize()
    else:
        casPreposition[preposition]=""
        
i=2
verbe_forme={}
for forme in morphosyntax["VER"]["FormesBase"]:
    verbe_forme[i]=forme
    i+=1

In [1216]:
#valAttribut

#### Remplacement des numéros de personnes pour les noms de macro LaTeX

In [1217]:
def remplacerPersonnes(chaine):
    chaine.replace("1","Un")
    return chaine

In [1218]:
remplacerPersonnes("1")

'1'

In [1219]:
def recoder(chaine,table):
    if type(chaine)==str:
        temp=unicode(chaine.decode('utf8')).translate(table)
        result=temp.encode('utf8')
    elif type(chaine)==unicode:
        result=chaine.translate(table)
    return result

accentedIn = unicode(phonology["translations"]["deaccent"]["in"])
deaccentIn = [ord(char) for char in accentedIn]
deaccentOut = unicode(phonology["translations"]["deaccent"]["out"])
deaccent = dict(zip(deaccentIn, deaccentOut))

deligatures=phonology["translations"]["deligatures"]

In [1220]:
syntagmes=morphosyntax["Syntagmes"]

In [1221]:
contractions=morphosyntax["Contractions"]
for contraction in contractions:
    temp=[]
    for element in contractions[contraction]:
        if isinstance(element,unicode):
            temp.append(element)
        else:
            temp.append(element.decode("utf8"))
    contractions[contraction]=temp

In [1222]:
syllabes=phonology["syllabes"]

In [1223]:
attributsFlexVerbe=[glosesVerbe for attributFlex in gloses["VER"] for glosesVerbe in gloses["VER"][attributFlex]]
attributsFlexVerbe

['PRS', 'PST', '3Sg', '3Du', '3Pl']

In [1224]:
def taches():
    '''
    seuil1 pour avoir une tâche
    seuil2 pour avoir plusieurs tâches
    '''
    seuil1=8
    seuil2=14
    def makeStain():
        seed=random.randint(1,1000)
        x=random.gauss(10,5)-1
        y=random.gauss(2,1)
        minimum=random.gauss(.2,.1)+.1
        maximum=random.gauss(.1,.05)+.5
        return "\\taches{%s}{%s}{%s}{%s}{%s}"%(seed,x,y,minimum,maximum)

    if print_taches:
        n=random.gauss(10,2.5)
        if n<seuil1:
            return ""
        elif n<=seuil2:
            return makeStain()
        else:
            nTaches=int(n-seuil1)
            stains=""
            for i in range(nTaches):
                stains+=makeStain()
            return stains
    else:
        return ""

In [1225]:
def faire_tableau(tableau,tab=(head,tail,"")):
    if len(tableau)==0: return
    comment=tab[2]
    for element in tab[0]:
        ajouterVocabulaire(comment+element)
    for element in tableau:
        ajouterVocabulaire(comment+element)
    for element in tab[1]:
        ajouterVocabulaire(comment+element)

In [1226]:
def print_tableaux(cols,recuTableau,texte="",debut=0,tab=(head,tail,""),boolEchantillon=True):
    tableau=sorted(list(set(recuTableau)))
    if debug: print "recuTableau",(recuTableau)
#    print (tableau)
    ajouterVocabulaire(tab[2]+"\\begin{multicols}{"+str(cols)+"}")
#    if texte!="":
    (table,reste)=filtrer_tableau(tableau,texte)
#    print reste
#    else:
#        (table,reste)=tableau
#        reste=[]
    chunk=(len(table)-debut*cols)/cols+1
    faire_tableaux(table,debut,cols,tab)
    ajouterVocabulaire(tab[2]+"\\end{multicols}")
    echantillon=min(len(reste),3)
    if echantillon>0 and boolEchantillon:
        for element in random.sample(reste,echantillon):
#            print element
            colonnes=element.split("&")
            graphie=colonnes[0].strip()
            phonologie=colonnes[1].strip()
            glose=colonnes[2].strip().strip("\\")
            prononciationExtrait.append(graphie+"&")
            prononciationExtrait.append("\\blanc{%s}"%phonologie+"&")
            prononciationExtrait.append("\\blanc{%s}\\\\"%glose)
            if debug: print "".join(prononciationExtrait)

In [1227]:
def faire_tableaux(tableau,debut=16,nombre=1,tab=(head,tail,"")):
    reste=[]
    if debug: print nombre,debut,tableau
    if debut!=0:
        for i in range(nombre):
            faire_tableau(tableau[debut*i:debut*(i+1)],tab)
        table=tableau[debut*nombre:]
    else:
        table=tableau
    longueur=len(table)
    chunk=longueur/nombre+1
    if debug: print "CHUNKING : ",longueur, nombre, chunk, table
    if chunk<maxChunk:
        chunks=chunk
    else:
        chunks=maxChunk
        reste=table[maxChunk*nombre:]
    if debug: print "RESTE : ", chunk, reste
    for i in range(nombre):
        faire_tableau(table[chunks*i:chunks*(i+1)],tab)
    if reste:
        faire_tableaux(reste,0,nombre,tab)

In [1228]:
def filtrer_tableau(tableau,filtre):
    presents=[]
    absents=[]
    for line in tableau:
        elements=line.split(" ")
        cleElement=elements[0].replace("\\","")
        if elements[0] in filtre:
            if not discriminant[cleElement] in discrimineur:
                discrimineur.append(discriminant[cleElement])
    #                print "présent",elements[0]
            presents.append(line)
        else:
#                print "absent",elements[0]
            absents.append(line)
    return (presents,absents)

In [1229]:
def faire_gn(depart,cas):
    global erg_genre, erg_nombre, abs_genre, abs_nombre
    if debug: print "groupe depart :", depart
    groupe_nom=[]
    groupe_nom.append(depart[0])
    if debug: print depart[0]
    for mot in depart[1:]:
        if debug: print mot
        if not "#" in mot:
            groupe_nom.extend(etendre_contraction([mot]))
    if debug: print "groupe nom :", groupe_nom
    mots=[]
    det=[]
    adj=[]
    nom=[]
    gp=[]
    structureSyntagme={key:[] for key in syntagmes['GN']}
    tete=""
    nombre=""
    classe=""
    classesNom=[]
    reste=0
    groupe_nom=[element for element in groupe_nom if element!=""]
    for mot in groupe_nom:
        if reste==0:
#            if mot=="deux" and "Du" in nombresNom:
            if mot=="deux" and "Du" in valAttributs["Nombre"]:
                nombre="DU"
                if det==[]: det.append(PFM.lexique.formeLexeme["des"][0])
            else:
                nomLexeme=PFM.lexique.formeLexeme[mot][0]
                categorie=PFM.lexique.lexemes[nomLexeme].classe.split(".")[-1]
                if debug or 0: 
                    print "mot",[mot]
                    print "vedette",PFM.lexique.formeLexeme[mot][0],categorie
                    print "categories",PFM.hierarchieCF.classes["NOM"],PFM.hierarchieCF.getCategory(categorie)
                if PFM.hierarchieCF.getCategory(categorie)=="NOM":
                    tete=categorie
                    if debug or 0: print "tête :", tete
                    tampon=tete.split('.')
#                    classe=tampon[0]
                    classesNom=PFM.lexique.formeLexeme[mot][0].split('.')[1:]
                    if debug or 0: print "classesNom",classesNom 
                    classe=[classeElement for classeElement in classesNom if classeElement in gloses["NOM"]["Genre"]][0]
                    if debug or 0: print "classes",mot,classe,classesNom
                    try:
                        typeMot=tampon[1]
                    except IndexError:
                        typeMot=''
                    if mot[len(mot)-1]=='s':
                        if nombre=="": nombre="PL"
                    else:
                        if nombre=="": nombre="SG"
                    nom.append(PFM.lexique.formeLexeme[mot][0])
#                    if typeCas=="NoCas":
#                        cas=""
                    if not "Cas" in boolAttribut["NOM"] or not boolAttribut["NOM"]["Cas"]:
                        cas=""
#                    else:
                    cellule=classe.capitalize()+nombre.capitalize()+cas.capitalize()
                    if cas=="ERG":
                        erg_genre=classe
                        erg_nombre=nombre
                        if debug: print "ERG",erg_genre, erg_nombre
                    elif cas=="ABS":
                        abs_genre=classe
                        abs_nombre=nombre
                        if debug: print "ABS",abs_genre, abs_nombre
                elif PFM.hierarchieCF.getCategory(categorie) in ["DET"]:
                    det.append(PFM.lexique.formeLexeme[mot][0])
#                elif PFM.hierarchieCF.getCategory(categorie) in ["ADJ"] or ("ADJ" in PFM.hierarchieCF.classes and categorie in PFM.hierarchieCF.classes["ADJ"]):
                elif PFM.hierarchieCF.getCategory(categorie) in ["ADJ"]:
                    adj.append(PFM.lexique.formeLexeme[mot][0])
                elif categorie=="PREP":			#Si on trouve une PREP, elle et le reste forment un GP
                    gp.append(mot)
                    reste=1
        else:							#On a trouvé une PREP, toute la suite va dans GP
            gp.append(mot)
    if debug: print "accord :", tete
    if reste==1: gp=faire_gp(gp)
    if debug: print "GP dans le GN : ", gp
    if debug: print "GN sans det ? ", det
    if not det: 
        if nom[0][0]==nom[0][0].upper():
            det.append(PFM.lexique.formeLexeme["les"][0])
        else:
            det.append(PFM.lexique.formeLexeme["des"][0])
    tempSyntagme=[]
    for mot in det:
        (casDet,nombreDet)=(cas,nombre)
        if not "Cas" in boolAttribut["DET"] or not boolAttribut["DET"]["Cas"]:
            casDet=""
        if not "Nombre" in boolAttribut["DET"] or not boolAttribut["DET"]["Nombre"]:
            nombreDet=""
        if not "Genre" in boolAttribut["DET"] or not boolAttribut["DET"]["Genre"]:
            classeDet=""
        else:
            classeDet=[classeElement for classeElement in classesNom if classeElement in gloses["DET"]["Genre"]][0]
#		glose=faire_glose(mot,classe,type,nombre)

        noligMot=mot.split(".")[0]
        for ligature in deligatures:
            noligMot=noligMot.replace(ligature,deligatures[ligature])
        ref="\\"+recoder(noligMot,deaccent).upper()
        for attributDet in morphosyntax["Attributs"]["DET"]:
            if attributDet=="Cas":
                ref+=casDet.capitalize()
            elif attributDet=="Nombre":
                ref+=nombreDet.capitalize()
            elif attributDet=="Genre":
                ref+=classeDet.capitalize()
        tempSyntagme.append(ref)
        lexemesLocaux.add(noligMot)
        texte.append(ref)
    structureSyntagme["DET"]=tempSyntagme
    mots.insert(syntagmes['GN'].index("DET"),separateurMots.join(structureSyntagme["DET"]))
    tempSyntagme=[]
    for mot in gp: 
#        mots.append(mot)
        tempSyntagme.append(mot)
    structureSyntagme["GP"]=tempSyntagme
    mots.insert(syntagmes['GN'].index("GP"),structureSyntagme["GP"])
    tempSyntagme=[]
    for mot in adj:
#		glose=faire_glose(mot,classe,type,nombre)
        (casAdj,nombreAdj)=(cas,nombre)
        if not "Cas" in boolAttribut["ADJ"] or not boolAttribut["ADJ"]["Cas"]:
            casAdj=""
        if not "Nombre" in boolAttribut["ADJ"] or not boolAttribut["ADJ"]["Nombre"]:
            nombreAdj=""
        if not "Genre" in boolAttribut["ADJ"] or not boolAttribut["ADJ"]["Genre"]:
            classeAdj=""
        else:
            classeAdj=[classeElement for classeElement in classesNom if classeElement in gloses["ADJ"]["Genre"]][0]

        noligMot=mot.split(".")[0]
        for ligature in deligatures:
            noligMot=noligMot.replace(ligature,deligatures[ligature])
            
        ref="\\"+recoder(noligMot,deaccent).lower()+classeAdj.capitalize()+nombreAdj.capitalize()+casAdj.capitalize()
#        mots.append(ref)
        lexemesLocaux.add(noligMot)
        texte.append(ref)
        tempSyntagme.append(ref)
    structureSyntagme["GADJ"]=tempSyntagme
    mots.insert(syntagmes['GN'].index("GADJ"),structureSyntagme["GADJ"])
    tempSyntagme=[]
#    print "classe,nombre,cas", classe, nombre, cas
    for mot in nom:
#		glose=faire_glose(mot,classe,type,nombre)

        noligMot=mot.split(".")[0]
        for ligature in deligatures:
            noligMot=noligMot.replace(ligature,deligatures[ligature])
            
        if mot.istitle():
            ref="\\"+recoder(noligMot,deaccent)+cellule
        else:
            ref="\\"+recoder(noligMot,deaccent).lower()+cellule
#        mots.append(ref)
        lexemesLocaux.add(noligMot)
        texte.append(ref)
        tempSyntagme.append(ref)
    structureSyntagme["NOM"]=tempSyntagme
    mots.insert(syntagmes['GN'].index("NOM"),structureSyntagme["NOM"])
    listeMots=[]
    for element in syntagmes['GN']:
        listeMots+=structureSyntagme[element]
    return (listeMots,(classesNom,nombre,cas))

In [1230]:
def faire_gp(groupe_prep,fonction=""):
    mots=[]
    groupe_prep=etendre_contraction(groupe_prep)
    if debug: print "faire_gp", groupe_prep
    formePreposition=groupe_prep[0]
    if debug: print [formePreposition], formePreposition
    preposition=PFM.lexique.formeLexeme[formePreposition][0]
#    if preposition!="à" or typeCas=="NoCas":
    if not "Cas" in boolAttribut["NOM"] or not boolAttribut["NOM"]["Cas"] or fonction!="IND":
        if debug: print u"fonction!=IND",[groupe_prep[0],u"à"]

        noligMot=groupe_prep[0]
        for ligature in deligatures:
            noligMot=noligMot.replace(ligature,deligatures[ligature])

        ref="\\"+recoder(noligMot,deaccent).upper()
        
        if debug: print ref
        if casSyntagmes and preposition in casPreposition:
            if debug: 
                print casPreposition, casPreposition[preposition]
            cas=casPreposition[preposition]            
            if "+" in casPreposition[preposition]:
                cas=cas.strip("+")
                mots.append(ref)
                lexemesLocaux.add(noligMot.upper())
                texte.append(ref)
        else:
            if debug: print "pas de Cas"
            cas=""
            mots.append(ref)
            lexemesLocaux.add(noligMot.upper())
            texte.append(ref)
        if debug:
            print "groupe prep :", groupe_prep
            print ref
        if len(groupe_prep)>1:
            groupe_nom=groupe_prep[1:]
            if debug: print groupe_nom[0]
            #
            # Choisir le cas en fonction de la préposition
            #
            (localMots,(localClasses,localNombre,localCas))=faire_gn(groupe_nom,cas)
            mots.insert(syntagmes['GP'].index("GN"),localMots)
        if debug: 
            print groupe_prep, cas, mots
        return mots
    elif fonction=="IND":
        groupe_nom=groupe_prep[1:]
        cas=casSyntagmes["IND"]
        if "+" in casSyntagmes["IND"]:
            
            noligMot=groupe_prep[0]
            for ligature in deligatures:
                noligMot=noligMot.replace(ligature,deligatures[ligature])
            ref="\\"+recoder(noligMot,deaccent).upper()
            cas=cas.strip("+")
            mots.append(ref)
            lexemesLocaux.add(noligMot.upper())
            texte.append(ref)
        if debug: print "faire_gn", groupe_nom, faire_gn(groupe_nom,cas)
        (localMots,(localClasses,localNombre,localCas))=faire_gn(groupe_nom,cas)
        mots.append(localMots)
        return mots

In [1231]:
def etendre_contraction(liste):
    result=[]
    if liste[0] in contractions.keys():
        if debug: print "EXT : ", liste, contractions[liste[0]],liste[1:] 
        result.extend(contractions[liste[0]])
        result.extend(liste[1:])
    else:
        result=liste
    return result

In [1232]:
def printflat(liste,suffixe="",prefixe=""):
    if debug: print "printflat", liste
    if not isinstance(liste, basestring):
        for element in liste:
            accumulerMots(prefixe)
            printflat(element,suffixe)
    else:
#        if "{preview}" in liste:
#            print "preview",prefixe,liste,suffixe
        accumulerMots(prefixe)
        accumulerMots(liste+suffixe)

In [1233]:
#######################
#
#	INITIALISATION DES VARIABLES
#
#######################

try:
    __IPYTHON__ 
    ipython=True
except: 
    ipython=False

In [1234]:
################
#
# LECTURE DU FICHIER DE LEXEMES
#
#		LES LIGNES QUI COMMENCENT PAR # SONT IGNOREES
#
################
texHeader=[]
version=theNotebook
texHeader.append("%% script : "+version)
texHeader.append('%%%% run : %s' % time.strftime("%y%m%d-%H%M"))

if ipython or True:
#    lexeme_nom=serie+"Lexemes.txt"
#    phrase_nom=serie+"Phrases.txt"
    phrase_nom=serie+complementPhrases+"Phrases.csv"
    traduction_nom=serie+complementPhrases+"Traductions.csv"
    ecriture_nom=serie+complementPhrases+"Ecrit.csv"
    ordre_nom=serie+complementPhrases+"Ordre.csv"
    mots_nom=serie+complementPhrases+"Mots.csv"
    cloze_nom=serie+complementPhrases+"Clozes.txt"
else:
    parser=optparse.OptionParser()
    parser.add_option("-o", "--out", dest="outfile", action="store_true", help="write to FILE")
    parser.add_option("-c", "--cloze", dest="print_cloze", action="store_true", help="write a CLOZE FILE")
    parser.add_option("-l", "--lexicon", dest="print_lexique", action="store_true", help="append a lexicon")
    parser.add_option("-r", "--roots", dest="print_racines", action="store_true", help="append a root list")

    (options, args) = parser.parse_args()
    lexeme_nom=args[0]
    phrase_nom=args[1]
    if len(args)>=3:
        traduction_nom=args[2]
    else:
        traduction_nom=""
    if len(args)>=4:
        ecriture_nom=args[3]
    else:
        ecriture_nom=""
    if len(args)>=5:
        ordre_nom=args[4]
    else:
        ordre_nom=""
    if len(args)>=6:
        mots_nom=args[5]
    else:
        mots_nom=""


#### Ouverture du fichier lexique

In [1235]:
def filtrerCloze(filtre):
    result=[]
    for ligne in clozeLines:
        if type(ligne)==str:
            ligne=unicode(ligne.decode('utf8'))
        ligne=ligne.strip()
        if not ligne.startswith("#"):
            clozeElements=ligne.split(";")
            if clozeElements[1] in filtre:
                result.append(ligne)
        else:
            result.append(ligne)
    return result

In [1236]:
def flatten(lis):
     for item in lis:
         if not isinstance(item, basestring):
             for x in flatten(item):
                 yield x
         else:        
             yield item

In [1237]:
previewerBegin="\\begin{preview}\\begin{flushleft}"
previewerEnd="\\end{flushleft}\\end{preview}"
def previewer(chaine,numLignesVides=0,suffixe=""):
    flatChaine=[mot+suffixe for mot in list(flatten(chaine))]
    result=previewerBegin+separateurMots.join(flatChaine)+"\\\\"*numLignesVides+previewerEnd
    return [result]

In [1238]:
def recoder(chaine,table):
    if type(chaine)==str:
        temp=unicode(chaine.decode('utf8')).translate(table)
        result=temp.encode('utf8')
    elif type(chaine)==unicode:
        result=chaine.translate(table)
    return result

In [1239]:
accentedIn = unicode(phonology["translations"]["deaccent"]["in"])
deaccentIn = [ord(char) for char in accentedIn]
deaccentOut = unicode(phonology["translations"]["deaccent"]["out"])
deaccent = dict(zip(deaccentIn, deaccentOut))

In [1240]:
tipaIn = unicode(phonology["translations"]["ipa"]["in"])
ipaIn = [ord(char) for char in tipaIn]
ipaOut = unicode(phonology["translations"]["ipa"]["out"])
toipa = dict(zip(ipaIn, ipaOut))

In [1241]:
#################################################
#################################################
#################################################
##
##
##	FAIRE LE TRI DES FORMES UTILISEES DANS LES PHRASES
##	AFFICHER DANS LES TABLEAUX SEULEMENT CES FORMES
##
##
#################################################
################################################
#
#
#	FAIRE LA LISTE DES PHRASES AVEC LES 4 LIGNES
#		GRAPHO, PHONO, GLOSE, TRAD
#
#
################################################
texte=[]
texteMots=set()
#graphies={}
#abs_genre=""
#abs_nombre=""
#PFM.lexique.formeLexeme

In [1242]:
def faire_phrases(phrase_file,sortie="latex"):
    print sortie
    phrasesLocales=phrase_file.readlines()
    numLignes=len(phrasesLocales)
    numImages=1
    numLignesVides=0
    if sortie in ["latex","traductions"]:
        finLigne="\\\\"
    elif sortie=="images":
        numImages=1
        numLignesVides=0
    else:
        finLigne="\n"
        numLignesVides=0

    debutLignePreview="\\begin{preview}"
    finLigneNoPreview="\\\\"*numLignesVides
    finLignePreview="\\\\"*numLignesVides+"\\end{preview}"
    numLignes=(len(phrasesLocales)//numImages)*numImages

    if print_phrases:
        comment=""
    else:
        comment="%"
    if sortie in ["latex","traductions"]: ajouterExemple("\\begin{phrases}")
    for nPhrase,line in enumerate(phrasesLocales[:numLignes]):
        #phrase contient une position par fonction
        #pour accueillir les équivalents kalabas des chunks français
        phrase=[0 for i in range(len(syntagmes['Phrase']))]
        tampon=(line.strip().rstrip('.')).replace("'"," ").split("\t")
    #    print tampon[0],type(tampon[0])
        if debug: print line
        if not tampon[0].startswith("#"):
            verbe=tampon[1].split(" ")
            verbeForme=verbe[0]
            if len(PFM.lexique.formeLexeme[verbeForme])!=1:
                print "FORME AMBIGUË", PFM.lexique.formeLexeme[verbeForme]
            verbeLexeme=PFM.lexique.formeLexeme[verbeForme][0]
            temp=verbeLexeme.split(".")
            if debug or 0: print temp
            formeCitation=temp[0]
            lexemesLocaux.add(formeCitation)
            for element in temp[1:]:
                if element in attributsFlexVerbe:
                    formeCitation+=element.capitalize()
            typeVerbe=temp[1]
            classeVerbe=""
            nombreVerbe=""
            nombrePersonne=""
    #        print verbeLexeme, formeCitation,typeVerbe
            verbeLemme="%s%s"%(formeCitation,typeVerbe.capitalize())
            verbeFormeIndex=PFM.lexique.lexemes[verbeLexeme].formes.index(verbeForme)
            if debug: print "verbe :", verbe
    #        if verbeLexeme.endswith("VI"):
            if casSyntagmes and "SUJ" in casSyntagmes and "Erg" in casSyntagmes["SUJ"]:
                if ".VI" in verbeLexeme:            
    #                suj_cas="Abs"
                    frSujetCas="Abs"
                    klbSujet="SUJ"
                else:
    #                suj_cas="Erg"
    #                obj_cas="Abs"
                    frSujetCas="Erg"
                    frObjetCas="Abs"
                    klbSujet="OBJ"
            elif casSyntagmes and "SUJ" in casSyntagmes and "Nom" in casSyntagmes["SUJ"]:
                frSujetCas=casSyntagmes["SUJ"]
                frObjetCas=casSyntagmes["OBJ"]
                klbSujet="SUJ"
            else:
                frSujetCas=""
                frObjetCas=""
                klbSujet="SUJ"
            suj_genre=valAttributs["Genre"][0]
            suj_nombre=valAttributs["Nombre"][0]
            obj_genre=valAttributs["Genre"][0]
            obj_nombre=valAttributs["Nombre"][0]
            abs_genre=valAttributs["Genre"][0]
            abs_nombre=valAttributs["Nombre"][0]
            sujet=tampon[0].strip().split(" ")
            (localMots,(localClasses,localNombre,localCas))=faire_gn(sujet,frSujetCas)
            if (debug or 0) and nPhrase==0: print (localMots,(localClasses,localNombre,localCas))
            if sortie=="ordre":
                phrase[syntagmes['Phrase'].index('SUJ')]=previewer(localMots,suffixe="{}")
            else:
                phrase[syntagmes['Phrase'].index('SUJ')]=localMots
            if debug: print localCas
            if not localCas or localCas in sujetVerbe:
                if localClasses and "Genre" in gloses["VER"]:
                    localClasse=[classeElement for classeElement in localClasses if classeElement in gloses["VER"]["Genre"]][0]
                else:
                    localClasse=""
                classeVerbe=localClasse
                nombreVerbe=localNombre
                nombrePersonne=localNombre
                casVerbe=localCas
                if (debug or 0) and nPhrase==0: print "frSuj",nombrePersonne
            if debug: print "sujet :",phrase[1]
            if len(tampon)>=3:
                if tampon[2] and tampon[2].startswith(","):
                    tampon[2]=tampon[2][1:]
                objet=tampon[2].split(" ")
                if debug: print "objet : ",objet
                if objet!=['']: 
                    (localMots,(localClasses,localNombre,localCas))=faire_gn(objet,frObjetCas)
                    if (debug or 0) and nPhrase==0: print ("frObj",(localClasses,localNombre,localCas),line)
                    if sortie=="ordre":
                        phrase[syntagmes['Phrase'].index('OBJ')]=previewer(localMots,suffixe="{}")
                    else:
                        phrase[syntagmes['Phrase'].index('OBJ')]=localMots
                    if debug or 0: print ("obj :",localCas, sujetVerbe)
                    if localCas in sujetVerbe:
                        if localClasses:
                            localClasse=[classeElement for classeElement in localClasses if classeElement in gloses["VER"]["Genre"]][0]
                        else:
                            localClasse=""
                        classeVerbe=localClasse
                        nombreVerbe=localNombre
                        nombrePersonne=localNombre
                        casVerbe=localCas
                        if (debug or 0) and nPhrase==0: print ("cas",nombrePersonne, line)
                elif klbSujet=="OBJ":
                    if boolAttribut["VER"]["Genre"]:
                        classeVerbe=morphosyntax["Defauts"]["Genre"]
                    if boolAttribut["VER"]["Nombre"]:
                        nombreVerbe=morphosyntax["Defauts"]["Nombre"]
                    if boolAttribut["VER"]["Pers"]:
                        nombrePersonne=morphosyntax["Defauts"]["NbPers"]
                    
            if len(tampon)>=4:
                if tampon[3] and tampon[3].startswith(","):
                    tampon[3]=tampon[3][1:]
                indirect=tampon[3].split(" ")
                if debug: print "indirect : ",indirect
                if indirect!=['']:
                    if sortie=="ordre":
                        phrase[syntagmes['Phrase'].index('IND')]=previewer(faire_gp(indirect,"IND"),suffixe="{}")
                    else:
                        phrase[syntagmes['Phrase'].index('IND')]=faire_gp(indirect,"IND")
            if len(tampon)>=5:
                if tampon[4] and tampon[4].startswith(","):
                    tampon[4]=tampon[4][1:]
                comp=tampon[4].split(" ")
                if comp!=['']:
                    if sortie=="ordre":
                        phrase[syntagmes['Phrase'].index('COMP')]=previewer(faire_gp(comp),suffixe="{}")
                    else:
                        phrase[syntagmes['Phrase'].index('COMP')]=faire_gp(comp)
            if len(tampon)>=6:
                if tampon[5] and tampon[5].startswith(","):
                    tampon[5]=tampon[5][1:]
                ajout=tampon[5].split(" ")
                if sortie=="ordre":
                    phrase[syntagmes['Phrase'].index('AJOUT')]=previewer(faire_gp(ajout),suffixe="{}")
                else:
                    phrase[syntagmes['Phrase'].index('AJOUT')]=faire_gp(ajout)
            if not boolAttribut["VER"]["Genre"]:
                classeVerbe=""
            if not boolAttribut["VER"]["Nombre"]:
                nombreVerbe=""
            if not boolAttribut["VER"]["Pers"]:
                nombrePersonne=""
            else:
                nombrePersonne="Trois"+nombrePersonne.capitalize()
            if (debug or 0) and nPhrase==0: print "accord Verbe",nombreVerbe, classeVerbe, sujetVerbe, line

            noligFormeCitation=formeCitation
            for ligature in deligatures:
                noligFormeCitation=noligFormeCitation.replace(ligature,deligatures[ligature])

            glose="\\"+recoder(noligFormeCitation,deaccent)\
                +morphosyntax["VER"]["FormesBase"][verbeFormeIndex].capitalize()\
                +nombrePersonne\
                +classeVerbe.capitalize()\
                +nombreVerbe.capitalize()
            if (debug or 0) and nPhrase<200: print "glose Verbe",glose
            if sortie=="ordre":
                phrase[syntagmes['Phrase'].index('VER')]=previewer([glose],suffixe="{}")
            else:
                phrase[syntagmes['Phrase'].index('VER')]=glose
            lexemesLocaux.add(noligFormeCitation)
            texte.append(glose)
            if sortie in ["latex","traductions"]: 
                ajouterExemple("\\ex")
                if print_glose and print_phono and print_ortho:
                    ajouterExemple(comment+"\\gloses")
                elif print_glose+print_ortho+print_phono==2:
                    ajouterExemple(comment+"\\gloses")
            for nMot,mot in enumerate(phrase):
                if mot!=0:
                    print syntagmes["Phrase"][nMot], mot
                    printflat(mot,"{}")
            print
            localAccumulateur=[accu for accu in accumulateur if accu!=""]
            if print_ortho:
                prefixe=""
                if sortie=="images":
                    if nPhrase%numImages==0:
                        prefixe="\\begin{preview}"
                    else:
                        prefixe=""
                    if nPhrase%numImages==numImages-1:
                        finLigne=finLignePreview
                    else:
                        finLigne=finLigneNoPreview
                elif sortie=="mots":
                    localAccumulateur=[debutLignePreview+accu+finLignePreview for accu in accumulateur if accu!=""]
            else:
                prefixe=marqueurCommentaire
            ajouterExemple(prefixe+separateurMots.join(localAccumulateur)+finLigne)
            for mot in phrase:
                if mot!=0:
                    printflat(mot,"P{}")
            if print_phono:
                prefixe=""
                if sortie in ["images","ordre"]:
                    prefixe=marqueurCommentaire
            else:
                prefixe=marqueurCommentaire
            ajouterExemple(prefixe+separateurMots.join(accumulateur)+finLigne)
            for mot in phrase:
                if mot!=0 and sortie in ["latex","traductions"]:
                    printflat(mot,"G{}")
            if print_glose:
                prefixe=""
                if sortie in ["images","ordre"]:
                    prefixe=marqueurCommentaire
            else:
                prefixe=marqueurCommentaire
            if sortie in ["latex","traductions"]: ajouterExemple(prefixe+separateurMots.join(accumulateur)+finLigne)
            if "," in line:
                consts=line.split("\t")
                disloc=[]
                canon=[]
                for const in consts:
                    if const.startswith(","):
                        disloc.append(const[1:])
                    else:
                        canon.append(const)
                line=",\t".join(disloc).strip()+", "+"\t".join(canon)
                        
            traduction=(line.strip().rstrip('.')).split()
            start=1
            for element in traduction:			# convertir les S majuscules à la finale des mots en minuscules
                element=element.strip("#")
                if element!="":
                    if start:
                        start=0
                        element=element.capitalize()
                    caracteres=element
                    if len(caracteres)>1 and caracteres!=caracteres.lower() and caracteres!=caracteres.capitalize():
                        caracteres=list(caracteres[0]+caracteres[1:].lower())
#                    accumulerMots("".join(caracteres).encode('utf8'))
                    accumulerMots("".join(caracteres))
            if sortie=="latex": 
                ajouterExemple(taches()+" ".join(accumulateur)+".")
            elif sortie=="images":
                ajouterExemple(marqueurCommentaire+"\\begin{preview}"+" ".join(accumulateur)+"."+"\\end{preview}")
            elif sortie in ["ordre","mots"]:
                ajouterExemple("\\begin{preview}"+" ".join(accumulateur)+"."+"\\end{preview}")
                #if sortie=="mots": ajouterExemple("\\begin{preview}"+"."+"\\end{preview}")
            else:
                if debug: print accumulateur
                ajouterExemple(" ".join(accumulateur)+".")
            del accumulateur[:]
            if sortie=="latex" and print_coffee and random.randint(1,4)==1:
                stain=random.choice(["A","B","C","D"])
                stain=random.choice(["A","B","C"])
                alpha=random.random()/1.5
                angle=random.randint(0,360)
                xoff=random.randint(-200,0)
#                ajouterExemple('\\\\\\cofe%sm{%.3f}{1}{%d}{%d}{0}' % (stain,alpha,angle,xoff))
                ajouterExemple('\\hspace{-.35\\textwidth}\\cofe%sm{%.3f}{1}{%d}{%d}{0}' % (stain,alpha,angle,xoff))

    if sortie in ["latex","traductions"]: ajouterExemple("\\end{phrases}")
    if sortie in ["latex","traductions"]:
        if ('options' in globals() and options.print_cloze) or print_lexique:
            tab=(head,tail,"")
        else:
            tab=(head,tail,"%")
#        prononciationExtrait=[]
        ajouterVocabulaire(tab[2]+"\\begin{itemize}")
        ajouterVocabulaire(tab[2]+"\\Needspace{8\\baselineskip}%")
        ajouterVocabulaire(tab[2]+"\\item NOMS\\\\[-3ex]")
        print_tableaux(dimensionsTableaux["NOM"]["cols"],tableaux["NOM"],texte,dimensionsTableaux["NOM"]["long"],tab,False)
        ajouterVocabulaire(tab[2]+"\\Needspace{8\\baselineskip}%")
        ajouterVocabulaire(tab[2]+"\\item ADJECTIFS\\\\[-3ex]")
        print_tableaux(dimensionsTableaux["ADJ"]["cols"],tableaux["ADJ"],texte,dimensionsTableaux["ADJ"]["long"],tab,False)
        ajouterVocabulaire(tab[2]+"\\Needspace{8\\baselineskip}%")
        ajouterVocabulaire(tab[2]+"\\item VERBES\\\\[-3ex]")
        print_tableaux(dimensionsTableaux["VER"]["cols"],tableaux["VER"],texte,dimensionsTableaux["VER"]["long"],tab,False)
#        print_tableaux(2,tableaux["VER"],texte,20,tab)
        ajouterVocabulaire(tab[2]+"\\Needspace{8\\baselineskip}%")
        ajouterVocabulaire(tab[2]+u"\\item DÉTERMINANTS\\\\[-3ex]")
        print_tableaux(dimensionsTableaux["DET"]["cols"],tableaux["DET"],texte,dimensionsTableaux["DET"]["long"],tab,False)
#        print_tableaux(3,tableaux["DET"],texte,0,tab)
        ajouterVocabulaire(tab[2]+"\\Needspace{8\\baselineskip}%")
        ajouterVocabulaire(tab[2]+u"\\item PRÉPOSITIONS\\\\[-3ex]")
        print_tableaux(dimensionsTableaux["PREP"]["cols"],tableaux["PREP"],texte,dimensionsTableaux["PREP"]["long"],tab,False)
#        print_tableaux(2,tableaux["PREP"],texte,0,tab)
        ajouterVocabulaire(tab[2]+"\\end{itemize}")

        if sortie=="latex":
            with codecs.open(serie+complementPhrases+"Exemples"+variante+".tex", 'wb',encoding="utf8") as output:
                for texHead in texHeader:
                    output.write(texHead+"\n")
                for exemple in exemples:
                    output.write(exemple+"\n")
        elif (sortie=="traductions" and variante=="-Corr"):
            with codecs.open(serie+complementPhrases+"Traductions"+variante+".tex", 'wb',encoding="utf8") as output:
                for texHead in texHeader:
                    output.write(texHead+"\n")
                for exemple in exemples:
                    output.write(exemple+"\n")

        if sortie=="latex":
            with codecs.open(serie+complementPhrases+"Vocabulaire"+variante+".tex", 'wb',encoding="utf8") as output:
                for texHead in texHeader:
                    output.write(texHead+"\n")
                for vocable in vocabulaire:
    #                print [vocable]
                    output.write(vocable+"\n")

            with codecs.open(serie+complementPhrases+"Prononciation"+variante+".tex", 'wb',encoding="utf8") as output:
                for texHead in texHeader:
                    output.write(texHead+"\n")
                for ligne in prononciationBegin+prononciationExtrait+prononciationEnd:
                    output.write(ligne+"\n")
#    elif sortie=="images":
#        with codecs.open(serie+complementPhrases+"Images"+variante+".tex", 'wb',encoding="utf8") as output:
#            for exemple in exemples:
#                output.write(exemple+"\n")
    elif sortie in ["images","ordre","mots"]:
        with codecs.open(serie+complementPhrases+sortie.capitalize()+variante+".tex", 'wb',encoding="utf8") as output:
            for texHead in texHeader:
                output.write(texHead+"\n")
            for exemple in exemples:
                output.write(exemple+"\n")
    if sortie in ["images","traductions","ordre","mots"]:
        clozeTraductions=[]
        clozeExemples=exemples[:]
        filtreLignes=["\\begin{phrases}","\\ex","\\gloses","\\end{phrases}"]
        if sortie=="traductions":
            clozeExemples=[exemple for exemple in clozeExemples if (not exemple in filtreLignes) and not exemple.startswith("%")]
        for orthoLigne,phonoLigne,tradLigne in zip(*[iter(clozeExemples)]*3):
            if sortie=="traductions" and 1:
                print "ortho",orthoLigne
                print "phono",phonoLigne
                print "trad",tradLigne
            phonoLigne=phonoLigne.replace(previewerBegin,"").replace(previewerEnd,"")
            phonoLigne=phonoLigne.replace("P{}"," ").replace("{}"," ")
            phonoLigne=phonoLigne.replace(debutLignePreview,"").replace(finLignePreview,"")
            phonoMots=phonoLigne.strip(marqueurCommentaire).replace(finLignePreview,"").strip().split()
            tradLigne=tradLigne.strip(marqueurCommentaire).replace(debutLignePreview,"").replace(finLignePreview,"")
            phonoPhrase=[]
            for element in phonoMots:
                cleElement=element.strip().strip("\\{}")
                if element=="\\ex": cleElement=""
                if cleElement:
                    if not cleElement in traductions: print cleElement,phonoMots
                    phonoPhrase.append(traductions[cleElement])
            if separateurPhonoCloze=="" : suppLigne=";"+" ".join(phonoPhrase)
            else: suppLigne=""
            result=separateurPhonoCloze.join(phonoPhrase)+";"+tradLigne+suppLigne
            clozeTraductions.append(result)
        with codecs.open(serie+complementPhrases+sortie.capitalize()+".txt", 'wb',encoding="utf8") as output:
            for ligne in clozeTraductions:
                output.write(ligne+"\n")

#### Traitement du fichier phrases

In [1243]:
sortie="latex"
lexemesLocaux=set()
try:
    phrase_file = codecs.open((phrase_nom),"r","utf8")
except IOError:
    print 'I could not open the sentence file', phrase_nom
    sys.exit()
exemples=[]
accumulateur=[]
vocabulaire=[]
prononciationExtrait=[]
faire_phrases(phrase_file,sortie=sortie)
phrase_file.close()

localClozes=filtrerCloze(lexemesLocaux)
with codecs.open(serie+complementPhrases+"Phrases-Clozes"+".txt", 'wb',encoding="utf8") as output:
    for ligne in localClozes:
        output.write(ligne+"\n")

latex
SUJ [u'\\DEFMSg', u'\\KalebMSg']
VER \etrePstTroisSg
OBJ [u'\\DEFMSg', u'\\chefMSg', u'\\DE', [u'\\INDMSg', u'\\villageMSg', u'\\petitMSg', u'\\DE', [u'\\INDFSg', u'\\foretFSg']]]

SUJ [u'\\DEFFSg', u'\\MahiraFSg']
VER \etrePstTroisSg
OBJ [u'\\DEFFSg', u'\\soeurFSg', u'\\grandFSg', u'\\DE', [u'\\DEFMSg', u'\\KalebMSg']]

SUJ [u'\\DEFFSg', u'\\VioletteFSg']
VER \etrePstTroisSg
OBJ [u'\\DEFFSg', u'\\filleFSg', u'\\DE', [u'\\DEFMSg', u'\\KalebMSg']]

SUJ [u'\\DEFFSg', u'\\MahiraFSg']
VER \chasserPstTroisSg
COMP [u'\\DANS', [u'\\DEFFSg', u'\\foretFSg', u'\\AVEC', [u'\\DEFFSg', u'\\filleFSg', u'\\DE', [u'\\DEFMSg', u'\\KalebMSg']]]]

SUJ [u'\\DEFFDu', u'\\filleFDu']
VER \voirPstTroisDu
OBJ [u'\\INDFSg', u'\\ombreFSg', u'\\grandFSg']

SUJ [u'\\DEMFSg', u'\\ombreFSg', u'\\rougeFSg', u'\\noirFSg']
VER \etrePstTroisSg
OBJ [u'\\INDMSg', u'\\demonMSg']

SUJ [u'\\DEFMSg', u'\\demonMSg', u'\\terribleMSg']
VER \devorerPstTroisSg
OBJ [u'\\INDMSg', u'\\villageoisMSg']
AJOUT [u'\\DERRIERE', [u'\\

In [1244]:
nomTableaux="Tableaux.yaml"
with open(serie+nomTableaux, 'r') as stream:
    tableaux=yaml.load(stream)

In [1245]:
sortie="traductions"
lexemesLocaux=set()
if traduction_nom.endswith("csv"):
    try:
        traduction_file = codecs.open(traduction_nom,"r","utf8")
    except IOError:
        print 'I could not open the translation file', traduction_nom
        sys.exit()      
    else:
        try:
            cloze_file = codecs.open(cloze_nom,"r","utf8")
        except IOError:
            print 'I could not open the cloze file', cloze_nom
            sys.exit()
        else:
            traductions={}
            for line in cloze_file.readlines():
                line=line.strip()
                if not line.startswith("#"):
                    elementsCloze=line.split(";")
                    traductions[elementsCloze[0]]=elementsCloze[5]
            cloze_file.close()
            exemples=[]
            accumulateur=[]
            vocabulaire=[]
#            prononciationExtrait=[]
            faire_phrases(traduction_file,sortie=sortie)
            traduction_file.close()

localClozes=filtrerCloze(lexemesLocaux)
with codecs.open(serie+complementPhrases+"%s-Clozes"%sortie.capitalize()+".txt", 'wb',encoding="utf8") as output:
    for ligne in localClozes:
        output.write(ligne+"\n")

traductions
SUJ [u'\\DEFMPl', u'\\chasseurMPl']
VER \offrirPrsTroisPl
OBJ [u'\\INDMDu', u'\\coyoteMDu']
IND [u'\\A', [u'\\DEFFSg', u'\\infirmiereFSg']]

SUJ [u'\\DEFFDu', u'\\fruitFDu']
VER \arriverPstTroisDu
COMP [u'\\DANS', [u'\\DEFFSg', u'\\cuisineFSg', u'\\DE', [u'\\DEFFSg', u'\\NicoleFSg']]]

SUJ [u'\\DEFFDu', u'\\tableFDu', u'\\petitFDu', u'\\jauneFDu']
VER \tomberPrsTroisDu
COMP [u'\\DEVANT', [u'\\DEFMPl', u'\\chatMPl', u'\\troisMPl', u'\\DE', [u'\\DEFMSg', u'\\NabilMSg']]]
AJOUT [u'\\DANS', [u'\\DEFFSg', u'\\cuisineFSg']]

SUJ [u'\\DEMMSg', u'\\chasseurMSg', u'\\grandMSg', u'\\DE', [u'\\INDMPl', u'\\coyoteMPl']]
VER \donnerPrsTroisSg
OBJ [u'\\INDFSg', u'\\maisonFSg']
IND [u'\\A', [u'\\DEFFPl', u'\\infirmiereFPl', u'\\DE', [u'\\DEFMSg', u'\\villageMSg']]]

SUJ [u'\\DEFMPl', u'\\villageMPl', u'\\troisMPl', u'\\DE', [u'\\DEFFSg', u'\\plaineFSg']]
VER \chasserPrsTroisPl
OBJ [u'\\DEFMPl', u'\\chasseurMPl', u'\\DE', [u'\\INDFPl', u'\\autrucheFPl']]
COMP [u'\\AVEC', [u'\\INDMPl', u'\\

In [1246]:
sortie="images"
lexemesLocaux=set()
if ecriture_nom.endswith("csv"):
    try:
        ecriture_file = codecs.open(ecriture_nom,"r","utf8")
    except IOError:
        print 'I could not open the file', ecriture_nom
        sys.exit()      
    else:
        try:
            cloze_file = codecs.open(cloze_nom,"r","utf8")
        except IOError:
            print 'I could not open the cloze file', cloze_nom
            sys.exit()
        else:
            ecriture={}
            for line in cloze_file.readlines():
                line=line.strip()
                if not line.startswith("#"):
                    elementsCloze=line.split(";")
                    ecriture[elementsCloze[0]]=elementsCloze[5]
            cloze_file.close()
            exemples=[]
            accumulateur=[]
            vocabulaire=[]
#            prononciationExtrait=[]
            faire_phrases(ecriture_file,sortie=sortie)
            ecriture_file.close()

localClozes=filtrerCloze(lexemesLocaux)
with codecs.open(serie+complementPhrases+"%s-Clozes"%sortie.capitalize()+".txt", 'wb',encoding="utf8") as output:
    for ligne in localClozes:
        output.write(ligne+"\n")

images
SUJ [u'\\DEFFSg', u'\\louveFSg']
VER \etrePrsTroisSg
COMP [u'\\AVEC', [u'\\DEFMSg', u'\\villageMSg']]

SUJ [u'\\DEFFSg', u'\\crocFSg']
VER \donnerPstTroisSg
OBJ [u'\\INDMSg', u'\\loupMSg', u'\\noirMSg']
IND [u'\\A', [u'\\DEMMDu', u'\\couteauMDu']]

SUJ [u'\\INDFDu', u'\\colereFDu']
VER \apporterPstTroisDu
OBJ [u'\\INDFPl', u'\\ombreFPl', u'\\blesseFPl']
IND [u'\\A', [u'\\DEMMPl', u'\\chefMPl']]

SUJ [u'\\DEMMPl', u'\\serpentMPl']
VER \tomberPrsTroisPl
COMP [u'\\SUR', [u'\\DEFFDu', u'\\filleFDu']]

SUJ [u'\\DEMFPl', u'\\mainFPl']
VER \jeterPstTroisPl
OBJ [u'\\DEFFDu', u'\\luneFDu', u'\\DE', [u'\\DEFMSg', u'\\villageMSg']]

SUJ [u'\\DEFFPl', u'\\foretFPl']
VER \passerPrsTroisPl
COMP [u'\\APRES', [u'\\DEMMPl', u'\\enfantMPl', u'\\differentMPl']]

SUJ [u'\\DEFMDu', u'\\criMDu']
VER \courirPstTroisDu
COMP [u'\\DERRIERE', [u'\\DEMMDu', u'\\arbreMDu', u'\\grandMDu']]

SUJ [u'\\DEFMPl', u'\\demonMPl']
VER \apporterPstTroisPl
OBJ [u'\\INDFPl', u'\\mainFPl']
IND [u'\\A', [u'\\DEFFDu', u'\

In [1247]:
sortie="ordre"
lexemesLocaux=set()
if ordre_nom.endswith("csv"):
    try:
        ordre_file = codecs.open(ordre_nom,"r","utf8")
    except IOError:
        print 'I could not open the file', ordre_nom
        sys.exit()      
    else:
        try:
            cloze_file = codecs.open(cloze_nom,"r","utf8")
        except IOError:
            print 'I could not open the cloze file', cloze_nom
            sys.exit()
        else:
            ordre={}
            for line in cloze_file.readlines():
                line=line.strip()
                if not line.startswith("#"):
                    elementsCloze=line.split(";")
                    ordre[elementsCloze[0]]=elementsCloze[5]
            cloze_file.close()
            exemples=[]
            accumulateur=[]
            vocabulaire=[]
#            prononciationExtrait=[]
            faire_phrases(ordre_file,sortie=sortie)
            ordre_file.close()

localClozes=filtrerCloze(lexemesLocaux)
with codecs.open(serie+complementPhrases+"%s-Clozes"%sortie.capitalize()+".txt", 'wb',encoding="utf8") as output:
    for ligne in localClozes:
        output.write(ligne+"\n")

ordre
SUJ [u'\\begin{preview}\\begin{flushleft}\\DEFMSg{} \\chasseurMSg{}\\end{flushleft}\\end{preview}']
VER [u'\\begin{preview}\\begin{flushleft}\\offrirPstTroisSg{}\\end{flushleft}\\end{preview}']
OBJ [u'\\begin{preview}\\begin{flushleft}\\INDMSg{} \\coyoteMSg{}\\end{flushleft}\\end{preview}']
IND [u'\\begin{preview}\\begin{flushleft}\\A{} \\DEFFSg{} \\NicoleFSg{}\\end{flushleft}\\end{preview}']

SUJ [u'\\begin{preview}\\begin{flushleft}\\DEFFDu{} \\autrucheFDu{}\\end{flushleft}\\end{preview}']
VER [u'\\begin{preview}\\begin{flushleft}\\dormirPrsTroisDu{}\\end{flushleft}\\end{preview}']
COMP [u'\\begin{preview}\\begin{flushleft}\\AVEC{} \\DEFFSg{} \\infirmiereFSg{} \\grandFSg{}\\end{flushleft}\\end{preview}']

SUJ [u'\\begin{preview}\\begin{flushleft}\\DEFMDu{} \\cafeMDu{} \\grandMDu{} \\noirMDu{}\\end{flushleft}\\end{preview}']
VER [u'\\begin{preview}\\begin{flushleft}\\tomberPrsTroisDu{}\\end{flushleft}\\end{preview}']
COMP [u'\\begin{preview}\\begin{flushleft}\\DANS{} \\DEFMSg{} 

In [1248]:
sortie="mots"
lexemesLocaux=set()
if mots_nom.endswith("csv"):
    try:
        mots_file = codecs.open(mots_nom,"r","utf8")
    except IOError:
        print 'I could not open the file', mots_nom
        sys.exit()      
    else:
        try:
            cloze_file = codecs.open(cloze_nom,"r","utf8")
        except IOError:
            print 'I could not open the cloze file', cloze_nom
            sys.exit()
        else:
            motsIsoles={}
            for line in cloze_file.readlines():
                line=line.strip()
                if not line.startswith("#"):
                    elementsCloze=line.split(";")
                    motsIsoles[elementsCloze[0]]=elementsCloze[5]
            cloze_file.close()
            exemples=[]
            accumulateur=[]
            vocabulaire=[]
#            prononciationExtrait=[]
            faire_phrases(mots_file,sortie=sortie)
            mots_file.close()

localClozes=filtrerCloze(lexemesLocaux)
with codecs.open(serie+complementPhrases+"%s-Clozes"%sortie.capitalize()+".txt", 'wb',encoding="utf8") as output:
    for ligne in localClozes:
        output.write(ligne+"\n")

mots
SUJ [u'\\DEFFSg', u'\\louveFSg']
VER \etrePrsTroisSg
COMP [u'\\AVEC', [u'\\DEFMSg', u'\\villageMSg']]

SUJ [u'\\DEFFSg', u'\\crocFSg']
VER \donnerPstTroisSg
OBJ [u'\\INDMSg', u'\\loupMSg', u'\\noirMSg']
IND [u'\\A', [u'\\DEMMDu', u'\\couteauMDu']]

SUJ [u'\\INDFDu', u'\\colereFDu']
VER \apporterPstTroisDu
OBJ [u'\\INDFPl', u'\\ombreFPl', u'\\blesseFPl']
IND [u'\\A', [u'\\DEMMPl', u'\\chefMPl']]

SUJ [u'\\DEMMPl', u'\\serpentMPl']
VER \tomberPrsTroisPl
COMP [u'\\SUR', [u'\\DEFFDu', u'\\filleFDu']]

SUJ [u'\\DEMFPl', u'\\mainFPl']
VER \jeterPstTroisPl
OBJ [u'\\DEFFDu', u'\\luneFDu', u'\\DE', [u'\\DEFMSg', u'\\villageMSg']]

SUJ [u'\\DEFFPl', u'\\foretFPl']
VER \passerPrsTroisPl
COMP [u'\\APRES', [u'\\DEMMPl', u'\\enfantMPl', u'\\differentMPl']]

SUJ [u'\\DEFMDu', u'\\criMDu']
VER \courirPstTroisDu
COMP [u'\\DERRIERE', [u'\\DEMMDu', u'\\arbreMDu', u'\\grandMDu']]

SUJ [u'\\DEFMPl', u'\\demonMPl']
VER \apporterPstTroisPl
OBJ [u'\\INDFPl', u'\\mainFPl']
IND [u'\\A', [u'\\DEFFDu', u'\\o

In [1249]:
#PFM.hierarchieCF.classes
#discrimineur
#tableaux["NOM"]

In [1250]:
faire_gn("des enfANTs dans les arbres".split(),"")

([u'\\INDMPl', u'\\enfantMPl', u'\\DANS', [u'\\DEFMPl', u'\\arbreMPl']],
 ([u'hyper', u'M'], 'PL', ''))

In [1251]:
PFM.lexique.formeLexeme[u"enfANT"]

[u'enfant.hyper.M']

In [1252]:
PFM.lexique.formeLexeme[u"enfants"][0]

u'enfant.F'

In [1253]:
#filtrerCloze(lexemesLocaux)